In [2]:
# Dependencies
import os
import numpy as np
import requests
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [3]:
#load 2019 june yellowcab trip data
data = pd.read_csv('yellow_tripdata_2019-06.csv')
tripdata=pd.DataFrame(data)

In [5]:
tripdata.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1,0.0,1,N,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1,4.4,1,N,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0,0.8,1,N,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1,1.7,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5


In [6]:
filterdata=tripdata[(pd.to_datetime(tripdata["tpep_pickup_datetime"])>"2019-06-22")&(pd.to_datetime(tripdata["tpep_pickup_datetime"])<"2019-07-01")]
filterdata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
599520,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,13.0,1.0,0.5,3.46,0.00,0.3,20.76,2.5
599521,2,2019-06-24 10:58:03,2019-06-24 11:08:39,1,1.63,1,N,230,142,1,8.5,1.0,0.5,1.50,0.00,0.3,14.30,2.5
599522,2,2019-06-24 11:14:35,2019-06-24 11:18:08,1,0.80,1,N,239,238,2,4.5,1.0,0.5,0.00,0.00,0.3,8.80,2.5
692228,2,2019-06-24 22:25:00,2019-06-24 22:40:32,6,8.53,1,N,237,138,1,25.0,0.5,0.5,6.98,6.12,0.3,41.90,2.5
694129,2,2019-06-24 22:59:13,2019-06-25 06:22:59,6,19.07,2,N,141,132,1,52.0,0.0,0.5,12.28,6.12,0.3,73.70,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6941019,1,2019-06-30 23:23:03,2019-06-30 23:39:48,1,0.90,1,N,68,158,1,11.0,3.0,0.5,2.00,0.00,0.3,16.80,2.5
6941020,1,2019-06-30 23:50:22,2019-06-30 23:57:01,1,0.50,1,N,246,90,2,6.0,3.0,0.5,0.00,0.00,0.3,9.80,2.5
6941021,1,2019-06-30 23:58:32,2019-07-01 00:00:42,1,0.20,1,N,90,186,1,3.5,3.0,0.5,1.45,0.00,0.3,8.75,2.5
6941022,2,2019-06-30 23:23:10,2019-06-30 23:30:45,1,1.38,1,N,140,163,1,7.5,0.5,0.5,2.26,0.00,0.3,13.56,2.5


In [7]:
# URL for GET requests to retrieve taxi zone matching data
with open('taxi_zones.json') as f:
    data = json.load(f)
zonedata=json.dumps(data)

In [8]:
data.keys()

dict_keys(['meta', 'data'])

In [9]:
zonedata={}
for zone in data["data"]:
    locid=int(zone[8])
    long,lati=zone[10].split(",")[1].split()
    lon=float(long)
    lat=float(lati)
    zonedata[locid]=(locid,lon,lat)
zonedata

{1: (1, -74.18448899999999, 40.69509499999987),
 2: (2, -73.82277105438692, 40.63557691408512),
 3: (3, -73.84725144800001, 40.870987051999926),
 4: (4, -73.97179142961485, 40.72581422114911),
 5: (5, -74.17348507999998, 40.56226748999995),
 6: (6, -74.06350565699998, 40.60215005399992),
 7: (7, -73.90325211599989, 40.767504322999905),
 8: (8, -73.92398142899997, 40.774615111999886),
 9: (9, -73.78486059299999, 40.76064436299992),
 24: (24, -73.96004456499999, 40.79804123499991),
 10: (10, -73.78234452799988, 40.68840743699995),
 11: (11, -74.00032353799993, 40.60262029199992),
 12: (12, -74.01539758699995, 40.704800148999894),
 13: (13, -74.01281566799992, 40.71735629799987),
 18: (18, -73.88593703500003, 40.86592439799995),
 25: (25, -73.98101786899988, 40.68893792699991),
 14: (14, -74.03284292006926, 40.64356371017527),
 15: (15, -73.77738937457792, 40.79659450183192),
 22: (22, -73.99009356099988, 40.62278895299992),
 23: (23, -74.19557688057968, 40.63500421857057),
 16: (16, -73.

In [10]:
zonedf=pd.DataFrame(zonedata).T
zonedf

,0,1,2
1,1.0,-74.184489,40.695095
2,2.0,-73.822771,40.635577
3,3.0,-73.847251,40.870987
4,4.0,-73.971791,40.725814
5,5.0,-74.173485,40.562267
...,...,...,...
256,256.0,-73.956810,40.712756
259,259.0,-73.852073,40.909197
260,260.0,-73.901471,40.759633
261,261.0,-74.013270,40.704553


In [11]:
zonedf.rename(columns={0:"locid",1:"lat",2:"lon"},inplace=True)
zonedf

,locid,lat,lon
1,1.0,-74.184489,40.695095
2,2.0,-73.822771,40.635577
3,3.0,-73.847251,40.870987
4,4.0,-73.971791,40.725814
5,5.0,-74.173485,40.562267
...,...,...,...
256,256.0,-73.956810,40.712756
259,259.0,-73.852073,40.909197
260,260.0,-73.901471,40.759633
261,261.0,-74.013270,40.704553


In [12]:
pkjoindata=filterdata.merge(zonedf,left_on="PULocationID",right_on="locid")
pkjoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid,lat,lon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,1.0,0.5,3.46,0.00,0.3,20.76,2.5,231.0,-74.008644,40.724221
1,2,2019-06-22 00:00:16,2019-06-22 00:42:16,1,10.68,1,N,231,39,1,...,0.5,0.5,0.00,6.12,0.3,44.92,0.0,231.0,-74.008644,40.724221
2,1,2019-06-22 00:25:43,2019-06-22 00:38:53,1,2.60,1,N,231,107,2,...,3.0,0.5,0.00,0.00,0.3,14.80,2.5,231.0,-74.008644,40.724221
3,1,2019-06-22 00:46:18,2019-06-22 01:03:02,2,3.90,1,N,231,80,2,...,3.0,0.5,0.00,0.00,0.3,18.80,2.5,231.0,-74.008644,40.724221
4,2,2019-06-22 00:04:09,2019-06-22 00:22:49,2,4.52,1,N,231,112,2,...,0.5,0.5,0.00,0.00,0.3,20.80,2.5,231.0,-74.008644,40.724221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991518,1,2019-06-30 19:02:10,2019-06-30 19:05:22,1,0.00,5,N,84,84,1,...,0.0,0.0,0.00,0.00,0.3,96.30,0.0,84.0,-74.159883,40.527369
1991519,2,2019-06-27 15:17:20,2019-06-27 15:17:25,0,0.00,5,N,172,172,1,...,0.0,0.5,14.79,0.00,0.3,88.75,0.0,172.0,-74.086913,40.568986
1991520,2,2019-06-27 22:36:46,2019-06-27 23:20:01,1,16.66,1,N,172,113,1,...,0.5,0.5,10.46,0.00,0.3,62.76,2.5,172.0,-74.086913,40.568986
1991521,2,2019-06-27 23:38:32,2019-06-28 00:21:58,5,10.60,1,N,105,43,1,...,0.5,0.5,8.46,0.00,0.3,50.76,2.5,105.0,-74.015404,40.693066


In [13]:
pkjoindata.rename(columns={"lat":"pklat","lon":"pklon"},inplace=True)
pkjoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid,pklat,pklon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,1.0,0.5,3.46,0.00,0.3,20.76,2.5,231.0,-74.008644,40.724221
1,2,2019-06-22 00:00:16,2019-06-22 00:42:16,1,10.68,1,N,231,39,1,...,0.5,0.5,0.00,6.12,0.3,44.92,0.0,231.0,-74.008644,40.724221
2,1,2019-06-22 00:25:43,2019-06-22 00:38:53,1,2.60,1,N,231,107,2,...,3.0,0.5,0.00,0.00,0.3,14.80,2.5,231.0,-74.008644,40.724221
3,1,2019-06-22 00:46:18,2019-06-22 01:03:02,2,3.90,1,N,231,80,2,...,3.0,0.5,0.00,0.00,0.3,18.80,2.5,231.0,-74.008644,40.724221
4,2,2019-06-22 00:04:09,2019-06-22 00:22:49,2,4.52,1,N,231,112,2,...,0.5,0.5,0.00,0.00,0.3,20.80,2.5,231.0,-74.008644,40.724221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991518,1,2019-06-30 19:02:10,2019-06-30 19:05:22,1,0.00,5,N,84,84,1,...,0.0,0.0,0.00,0.00,0.3,96.30,0.0,84.0,-74.159883,40.527369
1991519,2,2019-06-27 15:17:20,2019-06-27 15:17:25,0,0.00,5,N,172,172,1,...,0.0,0.5,14.79,0.00,0.3,88.75,0.0,172.0,-74.086913,40.568986
1991520,2,2019-06-27 22:36:46,2019-06-27 23:20:01,1,16.66,1,N,172,113,1,...,0.5,0.5,10.46,0.00,0.3,62.76,2.5,172.0,-74.086913,40.568986
1991521,2,2019-06-27 23:38:32,2019-06-28 00:21:58,5,10.60,1,N,105,43,1,...,0.5,0.5,8.46,0.00,0.3,50.76,2.5,105.0,-74.015404,40.693066


In [14]:
dojoindata=pkjoindata.merge(zonedf,left_on="DOLocationID",right_on="locid")
dojoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid_x,pklat,pklon,locid_y,lat,lon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,0.00,0.3,20.76,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
1,2,2019-06-22 00:14:36,2019-06-22 00:36:08,5,3.39,1,N,231,161,2,...,0.00,0.3,19.30,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
2,2,2019-06-22 03:06:17,2019-06-22 03:20:26,2,6.18,1,N,231,161,2,...,0.00,0.3,22.80,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
3,1,2019-06-22 06:48:30,2019-06-22 06:56:51,1,3.70,1,N,231,161,1,...,0.00,0.3,17.75,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
4,2,2019-06-22 07:01:28,2019-06-22 07:11:04,1,3.57,1,N,231,161,1,...,0.00,0.3,19.13,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980866,1,2019-06-25 16:28:29,2019-06-25 16:37:01,1,0.00,1,N,99,84,1,...,18.36,0.3,94.36,0.0,99.0,-74.168954,40.583098,84.0,-74.159883,40.527369
1980867,2,2019-06-30 01:45:46,2019-06-30 01:46:00,1,0.00,1,N,84,84,2,...,0.00,0.3,3.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369
1980868,2,2019-06-30 01:47:32,2019-06-30 01:47:36,4,0.00,5,N,84,84,1,...,0.00,0.3,108.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369
1980869,1,2019-06-30 19:02:10,2019-06-30 19:05:22,1,0.00,5,N,84,84,1,...,0.00,0.3,96.30,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369


In [15]:
dojoindata.rename(columns={"lat":"dolat","lon":"dolon"},inplace=True)
dojoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid_x,pklat,pklon,locid_y,dolat,dolon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,0.00,0.3,20.76,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
1,2,2019-06-22 00:14:36,2019-06-22 00:36:08,5,3.39,1,N,231,161,2,...,0.00,0.3,19.30,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
2,2,2019-06-22 03:06:17,2019-06-22 03:20:26,2,6.18,1,N,231,161,2,...,0.00,0.3,22.80,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
3,1,2019-06-22 06:48:30,2019-06-22 06:56:51,1,3.70,1,N,231,161,1,...,0.00,0.3,17.75,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
4,2,2019-06-22 07:01:28,2019-06-22 07:11:04,1,3.57,1,N,231,161,1,...,0.00,0.3,19.13,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980866,1,2019-06-25 16:28:29,2019-06-25 16:37:01,1,0.00,1,N,99,84,1,...,18.36,0.3,94.36,0.0,99.0,-74.168954,40.583098,84.0,-74.159883,40.527369
1980867,2,2019-06-30 01:45:46,2019-06-30 01:46:00,1,0.00,1,N,84,84,2,...,0.00,0.3,3.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369
1980868,2,2019-06-30 01:47:32,2019-06-30 01:47:36,4,0.00,5,N,84,84,1,...,0.00,0.3,108.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369
1980869,1,2019-06-30 19:02:10,2019-06-30 19:05:22,1,0.00,5,N,84,84,1,...,0.00,0.3,96.30,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369


In [16]:
pridedata=dojoindata[(pd.to_datetime(dojoindata["tpep_pickup_datetime"])>"2019-06-30")&(pd.to_datetime(dojoindata["tpep_pickup_datetime"])<"2019-07-01")]

In [17]:
pridedata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid_x,pklat,pklon,locid_y,dolat,dolon
707,2,2019-06-30 00:04:51,2019-06-30 23:58:15,1,6.00,1,N,231,161,2,...,0.00,0.3,22.30,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
708,2,2019-06-30 02:42:25,2019-06-30 03:04:07,1,3.53,1,N,231,161,1,...,0.00,0.3,24.96,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
709,2,2019-06-30 03:12:03,2019-06-30 03:26:06,1,3.36,1,N,231,161,1,...,0.00,0.3,20.16,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
710,2,2019-06-30 03:56:13,2019-06-30 04:10:38,1,3.48,1,N,231,161,1,...,0.00,0.3,21.00,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
711,2,2019-06-30 08:49:36,2019-06-30 09:06:54,2,6.78,1,N,231,161,1,...,0.00,0.3,24.80,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980856,1,2019-06-30 18:25:08,2019-06-30 19:29:41,1,35.10,1,N,132,84,1,...,12.24,0.3,127.84,0.0,132.0,-73.824861,40.663577,84.0,-74.159883,40.527369
1980860,1,2019-06-30 00:10:14,2019-06-30 01:13:21,2,29.50,1,N,164,84,1,...,18.36,0.3,126.16,2.5,164.0,-73.984075,40.743325,84.0,-74.159883,40.527369
1980867,2,2019-06-30 01:45:46,2019-06-30 01:46:00,1,0.00,1,N,84,84,2,...,0.00,0.3,3.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369
1980868,2,2019-06-30 01:47:32,2019-06-30 01:47:36,4,0.00,5,N,84,84,1,...,0.00,0.3,108.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369


In [18]:
priorwkdata=dojoindata[(pd.to_datetime(dojoindata["tpep_pickup_datetime"])>"2019-06-23")&(pd.to_datetime(dojoindata["tpep_pickup_datetime"])<"2019-06-24")]
priorwkdata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid_x,pklat,pklon,locid_y,dolat,dolon
54,1,2019-06-23 00:57:45,2019-06-23 01:15:02,1,3.70,1,N,231,161,1,...,0.0,0.3,21.95,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
55,2,2019-06-23 01:18:50,2019-06-23 01:29:39,1,3.05,1,N,231,161,1,...,0.0,0.3,17.76,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
56,2,2019-06-23 01:55:38,2019-06-23 02:05:13,2,2.96,1,N,231,161,1,...,0.0,0.3,17.16,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
57,2,2019-06-23 02:00:34,2019-06-23 02:13:55,5,3.89,1,N,231,161,1,...,0.0,0.3,20.16,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
58,1,2019-06-23 03:18:33,2019-06-23 03:29:30,1,2.80,1,N,231,161,1,...,0.0,0.3,17.15,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980803,1,2019-06-23 10:15:57,2019-06-23 10:36:28,1,4.70,1,N,178,111,1,...,0.0,0.3,18.80,0.0,178.0,-73.970472,40.624170,111.0,-73.992964,40.658337
1980808,1,2019-06-23 19:18:43,2019-06-23 19:56:59,1,13.40,1,N,132,154,1,...,0.0,0.3,44.80,0.0,132.0,-73.824861,40.663577,154.0,-73.934232,40.607898
1980824,2,2019-06-23 11:28:53,2019-06-23 12:16:25,1,21.64,1,N,4,154,1,...,0.0,0.3,76.56,2.5,4.0,-73.971791,40.725814,154.0,-73.934232,40.607898
1980828,2,2019-06-23 15:25:38,2019-06-23 15:40:53,1,2.94,1,N,21,154,2,...,0.0,0.3,14.30,0.0,21.0,-73.974031,40.608702,154.0,-73.934232,40.607898


In [27]:
# Create Engine
engine = create_engine("sqlite:///trip.db", pool_pre_ping=True)

In [28]:
pridedata.to_sql('pridedata', con=engine, if_exists='replace')

In [29]:
test=pd.read_sql("SELECT * FROM pridedata",engine)

In [30]:
test

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid_x,pklat,pklon,locid_y,dolat,dolon
0,707,2,2019-06-30 00:04:51,2019-06-30 23:58:15,1,6.00,1,N,231,161,...,0.00,0.3,22.30,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
1,708,2,2019-06-30 02:42:25,2019-06-30 03:04:07,1,3.53,1,N,231,161,...,0.00,0.3,24.96,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
2,709,2,2019-06-30 03:12:03,2019-06-30 03:26:06,1,3.36,1,N,231,161,...,0.00,0.3,20.16,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
3,710,2,2019-06-30 03:56:13,2019-06-30 04:10:38,1,3.48,1,N,231,161,...,0.00,0.3,21.00,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
4,711,2,2019-06-30 08:49:36,2019-06-30 09:06:54,2,6.78,1,N,231,161,...,0.00,0.3,24.80,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164226,1980856,1,2019-06-30 18:25:08,2019-06-30 19:29:41,1,35.10,1,N,132,84,...,12.24,0.3,127.84,0.0,132.0,-73.824861,40.663577,84.0,-74.159883,40.527369
164227,1980860,1,2019-06-30 00:10:14,2019-06-30 01:13:21,2,29.50,1,N,164,84,...,18.36,0.3,126.16,2.5,164.0,-73.984075,40.743325,84.0,-74.159883,40.527369
164228,1980867,2,2019-06-30 01:45:46,2019-06-30 01:46:00,1,0.00,1,N,84,84,...,0.00,0.3,3.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369
164229,1980868,2,2019-06-30 01:47:32,2019-06-30 01:47:36,4,0.00,5,N,84,84,...,0.00,0.3,108.80,0.0,84.0,-74.159883,40.527369,84.0,-74.159883,40.527369


In [31]:
priorwkdata.to_sql('priorwkdata', con=engine, if_exists='replace')

In [25]:
test2=pd.read_sql("SELECT * FROM priorwkdata",engine)

In [26]:
test2

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,locid_x,pklat,pklon,locid_y,dolat,dolon
0,54,1,2019-06-23 00:57:45,2019-06-23 01:15:02,1,3.70,1,N,231,161,...,0.0,0.3,21.95,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
1,55,2,2019-06-23 01:18:50,2019-06-23 01:29:39,1,3.05,1,N,231,161,...,0.0,0.3,17.76,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
2,56,2,2019-06-23 01:55:38,2019-06-23 02:05:13,2,2.96,1,N,231,161,...,0.0,0.3,17.16,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
3,57,2,2019-06-23 02:00:34,2019-06-23 02:13:55,5,3.89,1,N,231,161,...,0.0,0.3,20.16,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
4,58,1,2019-06-23 03:18:33,2019-06-23 03:29:30,1,2.80,1,N,231,161,...,0.0,0.3,17.15,2.5,231.0,-74.008644,40.724221,161.0,-73.975807,40.754677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197102,1980803,1,2019-06-23 10:15:57,2019-06-23 10:36:28,1,4.70,1,N,178,111,...,0.0,0.3,18.80,0.0,178.0,-73.970472,40.624170,111.0,-73.992964,40.658337
197103,1980808,1,2019-06-23 19:18:43,2019-06-23 19:56:59,1,13.40,1,N,132,154,...,0.0,0.3,44.80,0.0,132.0,-73.824861,40.663577,154.0,-73.934232,40.607898
197104,1980824,2,2019-06-23 11:28:53,2019-06-23 12:16:25,1,21.64,1,N,4,154,...,0.0,0.3,76.56,2.5,4.0,-73.971791,40.725814,154.0,-73.934232,40.607898
197105,1980828,2,2019-06-23 15:25:38,2019-06-23 15:40:53,1,2.94,1,N,21,154,...,0.0,0.3,14.30,0.0,21.0,-73.974031,40.608702,154.0,-73.934232,40.607898
